In [1]:
from typing import Optional, Any, Dict
import uuid
import copy
from dotenv import load_dotenv

load_dotenv()

from agent import create_graph
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow running asyncio in Jupyter
nest_asyncio.apply()

assistant_id = "E2j4gGtvsJ2v75THGY0L"

# Generate a unique thread ID for the entire conversation
thread_id = str(uuid.uuid4())
config = {
    "configurable": {
        "thread_id": thread_id,
        "model_name": "openai",
        "search_kwargs": {
            "namespace": f"{assistant_id}_DEVELOPMENT",
            "k": 5,
        },
    }
}
messages = []

async def process_user_input(user_input):
    messages.append(user_input)
    graph = await create_graph()

    state = {"messages": messages}

    async for event in graph.astream_events(state, config, version="v1"):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if isinstance(chunk.content, list):
                for item in chunk.content:
                    if isinstance(item, dict) and item.get("type") == "text":
                        print(item["text"], end="", flush=True)
            else:
                print(chunk.content, end="", flush=True)
        elif kind == "on_tool_start":
            print("\n--")
            print(
                f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
            )
        elif kind == "on_tool_end":
            print(f"Done tool: {event['name']}")
            print(f"Tool output was: {event['data'].get('output')}")
            print("--\n")
            
from langchain_core.messages import HumanMessage

async def main():
    try:
        while True:
            user_input = input("\nEnter your question (or 'exit' to quit): ")
            if user_input.lower() == "exit":
                break
            await process_user_input(HumanMessage(content=user_input))
    except RuntimeError as e:
        print(f"An error occurred: {e}")
        print(
            "Please ensure the AsyncConnectionPool is properly initialized in the agent.py file."
        )

# Use asyncio.get_event_loop().run_until_complete() instead of asyncio.run()
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


/Users/pascal/langgraph-example-pyproject/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
Task was destroyed but it is pending!
task: <Task pending name='pool-1-worker-0' coro=<AsyncConnectionPool.worker() done, defined at /Users/pascal/langgraph-example-pyproject/.venv/lib/python3.12/site-packages/psycopg_pool/pool_async.py:613> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task pending name='pool-1-worker-1' coro=<AsyncConnectionPool.worker() done, defined at /Users/pascal/langgraph-example-pyproject/.venv/lib/python3.12/site-packages/psycopg_pool/pool_async.py:613> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task pending name='pool-1-worker-2' coro=<AsyncConnectionPool.worker() done, defined at /Users/pascal/langgraph-example-pyproject/.venv

Tool Call: {'name': 'get_information', 'args': {'full_query': 'Wer ist Marc Langen?'}, 'id': 'call_WmaCjQL6q6GLdGE3kjqzm90p', 'type': 'tool_call'}
tenant siegburg 
docs[Document(metadata={'source': 'https://serviceportal.siegburg.de/Mitarbeiter/Marc_Langen'}, page_content='VOLLERNAME:\nMarc Langen\n\n TELEFON_GESCHAEFTLICH:\n+49 2241 102-1250\n\n TELEFON_MOBIL:\nNone\n\n EMAIL:\nMarc.Langen@Siegburg.de\n\n FAX:\nNone\n\n ABTEILUNG:\nStS Digitalisierung\n\n TITEL:\nSachbearbeiter\n\n ORGANISATION:\nKreisstadt Siegburg\n\n RAUM:\nNone\n\n'), Document(metadata={'parent_text': 'Frühjahrsputz zwischen Sieg und Agger\n2.400 Engagierte sammeln 35 Kubikmeter Müll\n18 März 2024 \n© Kreisstadt Siegburg\nStadt-Putz-Tag\nSiegburg. Auch wenn das Wetter am Samstagvormittag nicht unbedingt zum Aufenthalt im Freien einlud, griffen knapp 2.400 Kinder und Erwachsene zu Müllbeutel, Greifzange und Handschuh, um in Siegburg aufzuräumen. Rund 35 Kubikmeter Abfall kamen in wenigen Stunden zusammen. Darunter 